In [1]:
import psycopg2
import pandas as pd
import json
import pandas.io.sql as sqlio
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
import sklearn.metrics as metrics

In [2]:
def connect_to_db(database, hostname, port, userid, passwrd):
    # connects to the database
    conn_string = "host="+hostname+" port="+port+" dbname="+database+" user="+userid+" password="+passwrd
    conn=psycopg2.connect(conn_string)
    conn.autocommit = True
    cursor = conn.cursor()
    return conn, cursor

In [3]:

conn, cursor = connect_to_db("postgres", "classification-articles.cjgj2uy1bapa.us-east-1.rds.amazonaws.com", "5432", "classifyarticles", "hs2020")

In [4]:
def get_all_tables(cursor):
    cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
    return cursor.fetchall()

all_tables=get_all_tables(cursor)

In [5]:
len(all_tables)

569

Removed test_table and name_table as they aren't relevant

In [6]:
all_tables.remove(('name_table',))
all_tables.remove(('test_table',))

In [7]:
def find_table_name(lst, key):
    for i, dic in enumerate(lst):
        #print(i,dic)
        if key in dic.keys():
            return(list(dic.values()))
            #return dic[key]
    return -1

In [8]:
with open("iab_taxonomy-v2.json") as ifile:
    taxonomy = json.load(ifile)

In [9]:
def map_data(taxonomy, table_list):
    map_dict = {}
    for table_name in table_list:
        
        num = re.search(r"[0-9][0-9_]*", table_name[0])[0]
        #print(table_name, re.search(r"[0-9][0-9_]*", table_name[0]))
        val = num.replace("_", ".")
        name = find_table_name(taxonomy, val)
        tier_name = name
        if type(tier_name)==str:
            map_dict[table_name[0]]=tier_name
        elif type(tier_name)==int:
            continue
        else:
            while (type(tier_name) is dict):
                key = list(tier_name.keys())[0]
                tier_name = tier_name[key]
            map_dict[table_name[0]]=tier_name
    return map_dict

map_dict = map_data(taxonomy, all_tables)

In [10]:
len(map_dict)

550

In [11]:
import numpy as np
def get_table(table_name, conn,v):
    sql = "SELECT * FROM "+table_name+" ORDER BY RANDOM() LIMIT 100;" 
    dat = sqlio.read_sql_query(sql, conn)
    df=pd.DataFrame(dat)
    #print(v)
    df['hierarchy']=str(v)
    #print(df)
    return df

In [12]:
len(map_dict.keys())

550

In [13]:
import random


In [14]:
data = pd.DataFrame()
for k,v in list(sorted(map_dict.items()))[:194]:#list(map_dict.items())[:50]:
    data=pd.concat([data,get_table(k, conn,v)])

In [15]:
train_data = pd.DataFrame()
train_data=data.copy()

In [16]:
train_data.columns

Index(['index', 'id', 'content', 'title', 'keywords', 'description',
       'preprocessed_content', 'label', 'hierarchy', 'level_0'],
      dtype='object')

Column 'hierarchy_#tiers' specifies the number of tiers for that particular article ID

In [17]:
train_data['hierarchy_#tiers']=[len(i) for i in train_data['hierarchy'].str.split(":")]

Adding 4 columns one each for each tier (we have 4 tiers in our database)

In [18]:
train_data['tier1']=np.NaN
train_data['tier2']=np.NaN
train_data['tier3']=np.NaN
train_data['tier4']=np.NaN
train_data=train_data.reset_index(drop=True)

Storing each tier name in its appropriate column

In [19]:
#from tqdm.notebook import tqdm
for k in (range(len(train_data))):
    i=str(train_data.loc[k,'hierarchy'])
    i=i.split(":")       
    l=[re.sub(r'[{}\[\]]',"",x) for x in i]
    for j in range(1,len(l)+1):
        a="tier"+str(j)
        train_data.loc[k,a]=l[j-1]

Dropping unnecessary columns

In [ ]:
train_data=train_data.drop(['level_0'],axis=1)

In [26]:
train_data=train_data.drop(['keywords','description'],axis=1)

In [28]:
train_data.columns

Index(['index', 'id', 'content', 'title', 'preprocessed_content', 'label',
       'hierarchy', 'hierarchy_#tiers', 'tier1', 'tier2', 'tier3', 'tier4'],
      dtype='object')

In [27]:
train_data.head()

,index,id,content,title,preprocessed_content,label,hierarchy,hierarchy_#tiers,tier1,tier2,tier3,tier4
0,369,https://www.prnewswire.com/news-releases/dodge...,"AUBURN HILLS, Mich., Feb. 12, 2020 /PRNewswire...",Dodge Brand Launches New 'House of Power' Vide...,auburn hills mich feb prnewswire dodge launchi...,Automotive,['Automotive'],1,'Automotive',NaN,NaN,NaN
1,288,https://www.prnewswire.com/news-releases/autos...,"WEST MIDDLESEX, Pa., Feb. 14, 2020 /PRNewswire...",Autosoft Introduces Next-Gen All-in-One Techno...,west middlesex pa feb prnewswire autosoft prov...,Automotive,['Automotive'],1,'Automotive',NaN,NaN,NaN
2,545,https://www.prnewswire.com/news-releases/kia-w...,"CHICAGO, Feb. 6, 2020 /PRNewswire/ -- Today, a...",Kia Wins Two 2019 Motorweek Drivers' Choice Aw...,chicago feb prnewswire today chicago auto show...,Automotive,['Automotive'],1,'Automotive',NaN,NaN,NaN
3,397,https://www.prnewswire.com/news-releases/scani...,"SÖDERTÄLJE, Sweden, Feb. 12, 2020 /PRNewswire/...",Scania Deploys Battery Electric Trucks in Norway,dert lje sweden feb prnewswire norwegian whole...,Automotive,['Automotive'],1,'Automotive',NaN,NaN,NaN
4,89,https://www.prnewswire.com/news-releases/rollp...,"NEW YORK, Feb. 21, 2020 /PRNewswire/ -- Ride-o...",Rollplay Debuts Worldwide Exclusive MINI Coope...,new york feb prnewswire ride toy innovator rol...,Automotive,['Automotive'],1,'Automotive',NaN,NaN,NaN


In [29]:
train_data.sample(5)

,index,id,content,title,preprocessed_content,label,hierarchy,hierarchy_#tiers,tier1,tier2,tier3,tier4
14624,83,https://www.tmz.com/2019/03/02/jeremy-meeks-ch...,Jeremy Meeks has gone from the big house to th...,&#039;Hot Felon&#039;...,jeremy meeks gone big house dog house believe ...,Celebrity\ Homes,[{'Pop Culture': 'Celebrity Homes'}],2,'Pop Culture','Celebrity Homes',NaN,NaN
16809,69,https://www.motor1.com/news/376084/chevy-silve...,This week Chevy introduced an off-road-ready S...,Off-Road-Ready Chevy Silverado Could Preview H...,week chevy introduced road ready silverado bui...,Off\-Road\ Vehicles,[{'Automotive': {'Auto Body Styles': 'Off-Road...,3,'Automotive','Auto Body Styles','Off-Road Vehicles',NaN
8404,51,https://www.pluggedin.com/movie-reviews/commut...,The daily commute tends to be fairly predictab...,The Commuter,daily commute tends fairly predictable traveli...,Crime\ and\ Mystery\ Movies,[{'Movies': 'Crime and Mystery Movies'}],2,'Movies','Crime and Mystery Movies',NaN,NaN
15366,93,http://insideevs.com/vision-zero-zf-electric-f...,It takes a lot of work to build the future. In...,ZF’s Vision Zero Vehicle Aims For Zero Acciden...,take lot work build future automotive industry...,Driverless\ Cars,[{'Automotive': {'Auto Type': 'Driverless Cars...,3,'Automotive','Auto Type','Driverless Cars',NaN
16164,227,https://www.motor1.com/news/381333/audi-e-tron...,We first spied a hotter version of the Audi E-...,Audi E-Tron S Possibly Spied Testing At The Nu...,first spied hotter version audi e tron may spy...,SUV,[{'Automotive': {'Auto Body Styles': 'SUV'}}],3,'Automotive','Auto Body Styles','SUV',NaN
